# Imports and configuration.
todo: see how well the removing stopwords first does.

In [606]:
%matplotlib inline

In [607]:
import lda
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text

import common

import functions as f



In [608]:
matplotlib.style.use('ggplot')

# Load data

In [610]:
con = common.make_engine()

df_ep = pd.read_sql_table('recipes_recipe', con)
df_ep = df_ep[['title','ingredient_txt','url','image']]
df_ep['source'] = 'epicurious.com'
print('Loaded %s records from epicurious.com' % df_ep.shape[0])

df_ar = pd.read_sql_table('allrecipes', con)
df_ar = df_ar[['data-name','ingredients','url','data-imageurl']]
df_ar.columns = ['title','ingredient_txt','url','image']
df_ar['source'] = 'allrecipes.com'

df_ar = df_ar.drop_duplicates('url')
df_ar.reset_index()

print('Loaded %s records from allrecipes.com' % df_ar.shape[0])

df = pd.concat([df_ep, df_ar], ignore_index=True)

print('Loaded %s records in total' % df.shape[0])

## TODO: fix unicode here.



Loaded 16539 records in total


Loaded 4506 records from epicurious.com
Loaded 12033 records from allrecipes.com

## Examining and cleaning data

In [611]:
df = df[df['ingredient_txt'].str.len() > 20]
print df['ingredient_txt'].str.len().describe()
df['ingredient_txt'].str.len().plot(kind='hist').set_title('Ingredients character count')
df = df.reset_index()




count    16526.000000
mean       296.671366
std        127.103774
min         23.000000
25%        206.000000
50%        280.000000
75%        371.000000
max       1087.000000
Name: ingredient_txt, dtype: float64


In [612]:
## clean up quoting.
pattern = "[\"\']"
for k in ['title', 'ingredient_txt', 'url', 'image']:
    df[k] = df[k].str.replace(pattern, '')

## formatting ingredients.
df['ingredient_txt'] = df['ingredient_txt'].str.replace('\n',' ')


In [613]:
def rm_stopwords(stopwords, x):
    return ' '.join([w for w in x.replace('[\W]', ' ').split() if w not in stopwords])



In [675]:
df['ingredient_txt_no_stopwords'] = map(lambda x: rm_stopwords(f.get_stop_words(), x), df['ingredient_txt'])



In [676]:
# Extract features
# TODO: remove stopwords before vectorizing, because the stop words don't capture bigrams.

vectorizer = CountVectorizer(
stop_words='english'
, ngram_range=(1, 2)
, token_pattern='[A-Za-z]+'
, min_df = 10
, max_df = 0.25
)

features = vectorizer.fit_transform(df.ingredient_txt_no_stopwords)
## features is a document x term matrix.

wc = f.feature_counts(vectorizer, features)

wc.sort('count').tail(25).plot('word','count', kind='bar')


In [677]:
m = lda.LDA(n_topics=40, random_state=0, n_iter=200)
m.fit(features)
print('Finished running model')


Finished running model


# Evaluating the model.
## Convergence

In [678]:
plt.plot(m.loglikelihoods_, '-')
plt.title('Loglikelihood')


# Assessing topics

In [679]:
## Extracting topic data.
## most probable words by topic.
## TODO: check if these are properly sorted within each topic.
w = f.most_probable_words(m, vectorizer.get_feature_names(), 10)
w.columns = ['rank','topic','word','prob']

## most probable documents by topic.
# np.apply_along_axis(lambda i: df.iloc[i]['title'], 1, doc_ids)
doc_ids = np.argsort(m.doc_topic_, axis=0)[-4:-1,:].T
doc_probs = np.sort(m.doc_topic_, axis=0)[-4:-1,:].T


In [680]:

# I AM HERE: why are some stop words not beeing removed? like 'pound' in topic 3 below.

reload(f)
f.show_topics(m, df, doc_probs, doc_ids, w)


----------------------------------------------------------------------
 virgin  extra virgin  tomatoes    oil  vinegar  basil  olives   wine  virgin oil
  0.068         0.067     0.031  0.023    0.019  0.015   0.015  0.015       0.012
topic: 37
documents:
                     Roasted Pork Loin       0.74
 Spice Rubbed Pork Roast in Beer Gravy   0.721875
                Milk Braised Pork Loin  0.6863636
1 (2 1/2 pound) boneless pork loin roast, rolled tied 1 1/2 2 vegetable 2 butter 3 milk 1 rosemary 1 garlic, 3-----2 butter 2 garlic, 2 1 oregano 1/2 1/2 flakes 1/2 thyme 1/2 mustard 1 marjoram 1 basil 1 (3 pound) boneless, center-cut pork loin roast 1 1/2 16 beer 3 all-purpose flour 1 1/2 onion-----3 garlic, 1 rosemary 2 boneless pork loin roast 1/4 1/2 wine
----------------------------------------------------------------------
 chops  boneless  pork chops  vinegar  boneless pork  garlic  pound   loin   wine
 0.050     0.043       0.038    0.034          0.033   0.029  0.027  0.025  0.


topic: 33
documents:
        Easy Coq Au Vin  0.6066667
   Classic Veal Marsala   0.565625
 Hearts of Palm Risotto       0.55
1 butter 1 1/2 onion 2/3 Arborio rice 1/4 wine 3 boiling vegetable broth 1/2 hearts palm 1/4 Parmesan cheese 1 parsley 1 butter-----2 1 veal medallions 1 all-purpose flour 1 shallot, 1 mushrooms, 1 Marsala 1 garlic, 2 low-sodium chicken broth 1 low-sodium beef broth 2 unsalted butter-----1 vegetable 1 (4 pound) chicken, 1 1/4 1/4 1 1/2 wine 1 1/2 chicken stock 1 onion 1 cornstarch 1/3
----------------------------------------------------------------------
 broth  chicken broth  stock  sodium    low  vegetable  chicken stock  low sodium  sodium chicken
 0.130          0.096  0.036   0.035  0.035      0.028          0.027       0.027           0.019
topic: 34
documents:
 Asparagus and Mushroom Puff Pastry Pie  0.6238095
                        Vegetable Curry      0.605
                 Mom\u0027s Chicken Pie      0.604
1 spinach 4 cooked chicken, 1 mushrooms 1 so


topic: 29
documents:
    Bill\u0027s Beefuna Burgers  0.7318182
     Easy French Dip Sandwiches      0.705
 Easiest Slow Cooker French Dip  0.6863636
3 beef sirloin roast 1 (1 ounce) packet au jus mix 1 8 (1 ounce) provolone cheese 8 hoagie rolls, split-----1 (10.5 ounce) beef consomme 1 1 deli roast beef 8 provolone cheese 4 hoagie rolls, split-----1 1/2 round 1 (6 ounce) tuna oil, 1 sweet onion, 3 sweet pickle relish seasoning 6 hamburger buns, split
----------------------------------------------------------------------
 split   buns  roast  steak  hamburger  chuck  hamburger buns  pound  round
 0.030  0.028  0.027  0.025      0.021  0.020           0.019  0.017  0.016
topic: 30
documents:
      Slow Cooker Beer Pulled Pork  0.7346154
               Hunter\u0027s Roast      0.705
 Slow Cooker Root Beer Pulled Pork  0.6703704
1 (4 pound) pork shoulder roast 1 (12 fluid ounce) bottle root beer 1 (4 fluid ounce) bottle liquid smoke flavoring 4-----4 venison rump roast 2 (12 fluid ounce


topic: 25
documents:
            Penne with Shrimp  0.8304348
               Clams Italiano  0.8142857
 Fra Diavolo Sauce With Pasta  0.8115385
4 oil, 6 garlic, 3 tomatoes liquid, 1 1/2 1 flakes 1 (16 ounce) package linguine pasta 8 shrimp, deveined 8 bay scallops 1 parsley-----1/2 butter 5 garlic, 2 wine 1 oregano 1 parsley 1 flakes (optional) 36 clams shell, scrubbed-----1 (16 ounce) package penne pasta 2 1/4 onion 1 1/4 wine 2 (14.5 ounce) cans tomatoes 1 shrimp, deveined 1 Parmesan cheese
----------------------------------------------------------------------
 garlic  shrimp  package  ounce package  deveined  parmesan  parmesan cheese  tomatoes  shrimp deveined
  0.046   0.042    0.037          0.036     0.034     0.033            0.030     0.027            0.021
topic: 26
documents:
            Fallen Chocolate Cake  0.8916667
 Chocolate-Peanut Butter Fun Cake  0.8916667
            Homemade Marshmallows  0.8885714
Vegetable brushing pan About 1 confectioners sugar coating pan mar


topic: 21
documents:
         How to Cook a Turkey  0.7074074
         Heavenly Lamb Shanks  0.6928571
 Deep-Fried Turkey with Herbs  0.6894737
1 (12- 14- pound) turkey, neck giblets 4 5 gallons peanut canola 3 kosher 1 1/2 1 1/2 thyme 1 1/2 rosemary 1 1/2 paprika-----1/2 4 (1 pound) lamb shanks 2/3 all-purpose flour 1 leek, 1/2-inch 2 stalks celery, 2 carrots, 2 onions, chunks 12 garlic, unpeeled 1 bay leaf 1 peppercorns 1 sprig thyme 1 sprig rosemary 1 1/2 wine 3 chicken stock 1 Sea-----1 onion, 1 stalk celery, 1 carrot, 1 (12 pound) turkey, neck giblets 2 kosher 1 1 cayenne 3 butter 4 sprigs rosemary 1/2 sage
----------------------------------------------------------------------
 celery    bay  rosemary  sprigs  stalks  garlic   leaf  pound  bay leaf
  0.060  0.038     0.034   0.031   0.024   0.024  0.021  0.021     0.021
topic: 22
documents:
 Witchs Finger Bread Sticks with Maple Mustard Dip     0.605
                  Sweet Potato Shepherd\u0027s Pie     0.535
  Couscous with Roa

...


topic: 13
documents:
                      Pizza Pinwheels      0.805
 Spinach and Cheese Stuffed Manicotti  0.7864865
             Campfire Pepperoni Pizza    0.75625
1 refrigerated pizza dough 1/4 pizza sauce 1/2 mozzarella cheese 1/2 pepperoni-----2 (16 ounce) packages mozzarella cheese 28 manicotti shells 2 (16 ounce) packages frozen spinach, thawed 1 (16 ounce) jar Alfredo sauce 1 (8 ounce) package Parmesan cheese 2 (16 ounce) jars spaghetti sauce-----1 (8 ounce) refrigerated crescent roll dough 2 mozzarella cheese 24 pepperoni 1 (14 ounce) pizza sauce
----------------------------------------------------------------------
 ounce package  container  ounce container  mozzarella  ounce jar  spinach    jar  mozzarella cheese  pizza
         0.079      0.029            0.028       0.025      0.024    0.024  0.023              0.021  0.021
topic: 14
documents:
 Overnight Slow Cooker Oatmeal  0.8227273
              Homemade Granola  0.8185185
              A Better Granola  0.8184211
1


topic: 9
documents:
     Tuna Casserole III     0.8375
      Pot Roast in Foil  0.7947368
 Tater Tot Casserole II  0.7909091
1 beef 1 1 (32 ounce) package tater tots, thawed 1 (10.75 ounce) condensed cream mushroom soup 1 (10.75 ounce) condensed cream celery soup 1 (10.75 ounce) milk 1-----3 round 1 (10.75 ounce) condensed cream mushroom soup 1 packet onion soup mix 2-----2 (10.75 ounce) cans condensed cream mushroom soup 2 (6 ounce) cans tuna, flaked 7 potato chips
----------------------------------------------------------------------
 cream  condensed  ounce condensed  condensed cream  mushroom  mushroom soup  cream mushroom  ounce package  package
 0.063      0.057            0.043            0.043     0.033          0.030           0.028          0.026    0.023
topic: 10
documents:
          Cheese Lasagna  0.8741935
 Skillet Spinach Lasagna  0.8607143
               Manicotti  0.8555556
1 pint part-skim ricotta cheese 8 mozzarella cheese 3/4 Parmesan cheese 2 eggs 1 parsley 1 (16


topic: 5
documents:
  Leftover Turkey Cheesecake       0.57
   Luxe Truffle Deviled Eggs  0.5666667
 Devilish Green Eggs and Ham  0.5666667
1 dozen hard-cooked eggs-----1 dozen hard-cooked eggs-----1 leftover stuffing 1 leftover turkey gravy 4 cream cheese, softened 4 ricotta cheese 2 eggs 3 leftover turkey gravy 1 1/2 leftover roast turkey, and/or dark meat, 1 cranberry sauce
----------------------------------------------------------------------
 bacon  turkey  chipotle  yellow    mix  adobo  yellow onion   eggs  potatoes
 0.045   0.033     0.025   0.023  0.022  0.021         0.018  0.017     0.015
topic: 6
documents:
                           Pork Picadillo  0.7088235
 Cheese Ravioli with Three Pepper Topping  0.6703704
          Bob\u0027s Sweet Pepper Skillet      0.644
2 extra-virgin 2 sesame 3 green bell peppers, 1 yellow bell pepper, 1 bell pepper, 1 onion, 2 1/4 1/4-----1 cheese ravioli 3 1 onion, 1 green bell pepper, 1/2 bell pepper, 1/2 yellow bell pepper, 2 chicken broth, 


topic: 1
documents:
 Macaroni and Cheese Southern Style  0.7421053
      Microwave Macaroni and Cheese  0.7214286
          Quick Macaroni and Cheese  0.7190476
1 elbow macaroni 1/4 milk 1 (20 ounce) package frozen macaroni cheese 1/8 1 1/2 Cheddar cheese 3 Parmesan cheese-----8 macaroni 2 butter 2 1/2 onion 16 cubed processed cheese food 3/4 milk-----1 (7.25 ounce) package macaroni cheese mix 1 1/4 evaporated milk 2 eggs 8 Cheddar cheese, 1/2 butter, -
----------------------------------------------------------------------
 cheddar cheese   milk  cubed  cooked  sharp  ounce package  sharp cheddar  package  macaroni
          0.082  0.040  0.033   0.031  0.025          0.023          0.023    0.023     0.023
topic: 2
documents:
                   Asian Barbecue Burgers  0.8115385
 Super-Simple, Super-Spicy Mongolian Beef  0.8115385
                           Spicy Eggplant  0.7965517
2 vegetable 4 Japanese eggplants, 1-inch 2 vegetable 2 onions, 1 2 soy sauce 2 1 1/2 oyster sauce 1 chi

topic: 0
documents:
        County Fair Corn Dogs       0.77
    Brown Diner Fried Chicken  0.7421053
 German-Russian-Dakota Knefla    0.69375
2 all-purpose flour 1 baking 1 1/4 1 egg 1/4 milk 2 vegetable 3 potatoes,-----3 peanut frying 2 all-purpose flour 1 onion 1 paprika, 1 1/2 milk 1 egg 1 fryer chicken,-----10 beef frankfurters 1/2 all-purpose flour dusting 2/3 cornmeal 1/3 all-purpose flour 2 corn flour (optional) 1 1 cayenne 1 egg 2 corn 1/2 1 quart vegetable frying
----------------------------------------------------------------------
 purpose  purpose flour   milk  vegetable  frying  butter purpose   eggs  flour milk    egg
   0.136          0.134  0.036      0.033   0.026           0.019  0.019       0.014  0.014

In [276]:
# Plotting word distributions for each topic.
wb = f.most_probable_words(m, vectorizer.get_feature_names(), 10)
wb.columns = ['rank','topic','word','prob']

## make figure of word distributions for each topic.
g = sns.FacetGrid(wb, col='topic', col_wrap=10)
p = g.map(sns.barplot, 'word', 'prob')
## save figure for easier viewing.
p.savefig('word_dist_by_topic.png')

## TODO: figure out way of examining probs of words in relation to topic coherence:
## high average prob?

## make figure of document distributions for each topic.
## for each topic, show distribution of documents.





In [ ]:
## examine topics:
## 14: one very probable word.
## 32: many very probable words.
## 30: no very probable words.




In [144]:
## TODO: store one set of results for each run.


## massage document ids and probabilities into form suitable for database.
di = pd.DataFrame(doc_ids)
di['topic'] = di.index
di = pd.melt(di, id_vars='topic')
di.columns = ['topic','rank','recipe_key']

dp = pd.DataFrame(doc_probs)
dp['topic'] = dp.index
dp = pd.melt(dp, id_vars='topic')
dp.columns = ['topic','rank','prob']

dd = pd.merge(di, dp)
dd.to_sql('doc_prob', con, if_exists='replace')

# store recipes
df['key'] = df.index
df.to_sql('clean_recipes', con, if_exists='replace')

# store words
w.columns = ['rank','topic','word','prob']
w.to_sql('word_probs', con, if_exists='replace')


In [285]:

xx = pd.merge(df, dd, left_on='key', right_on='recipe_key', how='right')

## topics with low word probs.
## but, they seem pretty good.
print 'topics with low word probs.'
for n, g in xx[xx['topic'].isin([5,8,16,18,30])].groupby('topic'):
    print 'topic: %s' % n
    print g[['title','prob']].sort('prob').to_string()

##
print '='*80
print 'topics with one high word prob.'
print '='*80
for n, g in xx[xx['topic'].isin([1,4,9,14,21])].groupby('topic'):
    print 'topic: %s' % n
    print g[['title','prob']].sort('prob').to_string()




topics with low word probs.
topic: 5
                                          title      prob
31                       Fruit and Nut Pancakes  0.816667
27   Coconut Macaroon Sandwiches with Lime Curd  0.821212
116                        Joyce\u0027s Granola  0.823077
topic: 8
                                        title      prob
71      Spicy Coconut and Lime Grilled Shrimp  0.638636
98  Garlic, Cilantro, and Lime Sauteed Shrimp  0.669697
51                  Spicy Lime Grilled Shrimp  0.684000
topic: 16
                                                title      prob
25       Chocolate-Dipped Salted Caramel Marshmallows  0.765278
34  Maple-Gingerbread Layer Cake with Salted Maple...  0.811538
5                        Bourbon-Vanilla Marshmallows  0.816279
topic: 18
                                title      prob
78          Rajma (Kidney Bean Curry)  0.743299
43  Indian Chicken Curry (Murgh Kari)  0.753608
55           Indian Dahl with Spinach  0.762000
topic: 30
                    

In [469]:
## relationship between doc prob and length:
dpa = pd.DataFrame({'max_prob':np.max(m.doc_topic_, axis=1), 'topic':np.argmax(m.doc_topic_, axis=1)})
dpa = df.join(dpa)
dpa['ingredient_len'] = dpa['ingredient_txt'].str.len()

dpa.plot('ingredient_len', 'mdpa_prob', kind='scatter')

# g = sns.FacetGrid(dpa, col='topic', col_wrap=10)
# p = g.map(sns.barplot, 'word', 'prob')
# ## save figure for easier viewing.
# p.savefig('word_dist_by_topic.png')



In [286]:
show_topics(m, df, doc_probs, w)



topic: 39
documents:
        Sesame Seared Tuna  0.8204545
 Japanese Broiled Mackerel  0.8096774
          Kung Pao Chicken      0.805
 1 tablespoon soy sauce 2 teaspoons Chinese rice wine or dry sherry 1 1/2 teaspoons cornstarch -----4 mackerel fillets 1/4 cup soy sauce 1/4 cup mirin (Japanese sweet wine) 1 tablespoon white sugar 1/2 tablespoon grated fresh ginger root-----1/4 cup soy sauce 1 tablespoon mirin (Japanese sweet wine) 1 tablespoon honey 2 tablespoons sesame oil 1 tablespoon rice wine vinegar 4 (6 ounce) tuna steaks 1/2 cup sesame seeds wasabi paste 1 tablespoon olive oil
----------------------------------------------------------------------
 soy sauce  minced  ginger  sesame  sugar  teaspoons  fresh ginger  cup soy  sauce tablespoons
     0.040   0.022   0.020   0.020  0.016      0.014         0.013    0.012              0.012



topic: 35
documents:
                                            BLT  0.8227273
 Amy\u0027s Triple Decker Turkey Bacon Sandwich       0.77
                         Grilled Cheese De Mayo  0.7666667
1 tablespoon mayonnaise, divided 2 slices white bread 2 slices American cheese 1 slice pepperjack cheese-----2 slices white bread 1/4 cup mayonnaise 3 lettuce leaves 1 tomato, thinly sliced 3 slices turkey bacon 3 slices Cheddar cheese-----4 slices bacon 2 leaves lettuce 2 slices tomato 2 slices bread, toasted 1 tablespoon mayonnaise
----------------------------------------------------------------------
 bacon  bread  slices bacon  split  mayonnaise    ham   inch  cooked  inch slices
 0.042  0.036         0.018  0.014       0.012  0.012  0.012   0.011        0.010
topic: 36
documents:
        Whole Wheat Pasta Marinara  0.7527778
 Penne with Pancetta and Mushrooms  0.7468085
                       Pastachutta  0.7178571
1 (16 ounce) package spaghetti 1/4 cup butter 4 cloves garlic, thinly s


topic: 31
documents:
                                    Sauerbraten IV  0.5953125
 Herby Black Rice Salad with Radishes and Ricot...  0.5695652
                   Simple Garlic Confit with Herbs    0.56875
 9–10 heads of garlic (about 3 cups cloves) 9–10 sprigs of thyme or another woody herb like rosemary  3 cups olive oil ----- 1/4 cup olive oil 2 tablespoons Sherry vinegar or red wine vinegar Kosher salt, freshly ground pepper -----1 1/2 cups red wine vinegar 1/2 cup dry red wine 2 onions, sliced 2 carrots, sliced 1 bay leaf 1/8 teaspoon whole allspice 3 whole cloves 1 tablespoon black peppercorns 1 tablespoon salt 4 pounds rump roast 1/2 cup butter 1 tablespoon vegetable oil 5 tablespoons all-purpose flour 1 tablespoon white sugar 2/3 cup crushed gingersnap cookies
----------------------------------------------------------------------
   red  vinegar  red wine    bay  wine vinegar  cloves  leaves  white  sprigs
 0.041    0.036     0.031  0.031         0.027   0.025   0.023  0.021 


topic: 27
documents:
       Venison Salisbury Steak  0.5516667
         Cube Steak with Gravy  0.5038462
 Simple Classic Roasted Turkey  0.4789474
1 (10 pound) whole turkey, neck and giblets optional salt and ground black pepper to taste 1/3 cup water, or as needed-----1/4 cup flour 1/4 cup cornstarch 1 teaspoon ground black pepper 1/2 teaspoon salt 6 cube steaks 2 tablespoons butter, or as needed 1 cup milk, or to taste salt and ground black pepper to taste-----1/4 cup oil 1 pound venison, cut into cubes 1 (16 ounce) package buttery round crackers, crushed fine 1/4 teaspoon garlic powder, or to taste onion salt, or to taste salt and pepper to taste 2 cubes beef bouillon 1 cup boiling water 1 egg 1 medium onion, sliced 1 (4.5 ounce) can sliced mushrooms 1 tablespoon cornstarch 1/4 cup cold water
----------------------------------------------------------------------
 vegetable  vegetable oil  needed  pepper taste  cup water  tablespoon vegetable  optional  tablespoons vegetable  bouill


topic: 23
documents:
                         Savory Apple Pork Chops  0.7947368
                          Apple Butter Pork Loin  0.7717949
 Bourbon Apple Cider and Honey Glazed Pork Chops  0.7457143
2 cups apple cider 1/2 cup honey 1/4 cup apple cider vinegar 2 tablespoons brown sugar 1/3 cup whiskey (such as Jack Daniels®) 3 tablespoons whiskey (such as Jack Daniels®) 4 thick center-cut pork chops-----2 (2 pound) boneless pork loin roast seasoning salt to taste 2 cups apple juice 1/2 cup apple butter 1/4 cup brown sugar 2 tablespoons water 1/4 teaspoon ground cinnamon 1/4 teaspoon ground cloves-----4 pork chops salt and pepper to taste 2 apples, sliced thin 1/4 cup ketchup 3 tablespoons brown sugar
----------------------------------------------------------------------
 brown  brown sugar   pork  vinegar  apple  white  chops  cider  pork chops
 0.045        0.043  0.037    0.028  0.022  0.022  0.020  0.015       0.015
topic: 24
documents:
  Whole Chicken Slow Cooker Recipe  0.752777

...


topic: 15
documents:
 Chicken Pecan Quiche  0.5978723
       Crab Quiche II  0.5923077
   Quiche Lorraine II  0.5923077
1 (9 inch) deep dish pie crust 2 eggs, beaten 1/2 cup milk 1/2 cup mayonnaise 1 teaspoon cornstarch 1 1/2 cups shredded Swiss cheese 1/2 pound imitation crabmeat 1 pinch ground black pepper-----1 recipe pastry for a 9 inch single crust pie 6 slices bacon 1 onion, sliced 3 eggs, beaten 1 1/2 cups milk 1/4 teaspoon salt 1 1/2 cups shredded Swiss cheese 1 tablespoon all-purpose flour-----1 cup chopped, cooked chicken 1 cup shredded Swiss cheese 1/4 cup chopped onion 1 tablespoon all-purpose flour 1/2 cup chopped pecans 1 (9 inch) unbaked deep-dish pastry shell 2 eggs, beaten 1 cup 2% milk 1/2 teaspoon brown mustard
----------------------------------------------------------------------
 beaten  eggs beaten  shredded   milk  cooked   inch    pie  diced  teaspoon salt
  0.048        0.025     0.024  0.023   0.020  0.018  0.018  0.017          0.015
topic: 16
documents:
   


topic: 11
documents:
   Quinoa Stuffed Peppers  0.6741379
 Superb Sausage Casserole  0.6346939
                     Tava  0.6089286
2 pounds beef stew meat, cut into 1/2 inch pieces 1 small red onion, diced 1 medium green bell pepper, seeded and diced 2 fresh tomatoes, diced 1 green chile pepper, seeded and diced (optional) 1/2 teaspoon chile paste-----6 links pork sausage 1 large red bell pepper, chopped 1 large orange bell pepper, chopped 1 large yellow bell pepper, chopped 1 large onion, diced 1/2 cup ketchup 3/4 cup hot water salt and pepper to taste-----1 cup quinoa, rinsed and drained 2 cups water 2 tablespoons olive oil 1 small onion, diced 2 cloves garlic, minced 1 zucchini, diced 1 small eggplant, diced 1 tomato, diced 1 cup tomato sauce salt and ground black pepper to taste 6 bell peppers, tops cut off and seeded 1 cup shredded mozzarella cheese, or more to taste
----------------------------------------------------------------------
 bell pepper  diced  green    red  green b


topic: 7
documents:
 Fresh Herb, Grapefruit, and Fennel Salad     0.6275
                 Caprese-Stuffed Zucchini  0.5772727
          Kale Pesto with Toasted Walnuts  0.5673913
 2 cups packed torn kale leaves, stems removed 1 cup packed fresh basil leaves 1 teaspoon sea salt 1/4 cup extra virgin olive oil 1/4 cup toasted walnuts 4 cloves garlic, chopped 1/2 cup grated Parmesan cheese -----2 large zucchinis, halved lengthwise and seeded sea salt to taste freshly ground black pepper to taste extra-virgin olive oil, or to taste 2 (1/2 inch thick) slices eggplant 1 cup cherry tomatoes, quartered 1 clove garlic, minced 1 cup fresh basil leaves, divided 1 teaspoon sea salt 1/2 teaspoon ground black pepper 3 tablespoons extra-virgin olive oil, plus more for drizzling 1 tablespoon balsamic vinegar----- 1 large ruby-red grapefruit or 3 blood oranges 2 small fresh fennel bulbs, trimmed, halved vertically, sliced paper-thin 1 cup fresh Italian parsley leaves 1/4 cup fresh mint leaves Extra-vir


2 cups rice-bread crumbs (such as PaneRiso™) 1/2 cup olive oil 1 teaspoon dried oregano 1 teaspoon dried basil 1 teaspoon dried rosemary 1 teaspoon garlic powder 2 pork tenderloins-----1 (4 pound) frozen duck 6 tablespoons olive oil 2 cups dry sherry 3 teaspoons dried oregano 3 teaspoons dried rosemary 3 teaspoons dried basil-----1 lemon 2 teaspoons dried oregano 3 cloves garlic, minced 1 tablespoon olive oil 1/4 teaspoon salt 1/4 teaspoon ground black pepper 6 chicken legs
----------------------------------------------------------------------
 teaspoon dried  oregano  dried oregano  teaspoons  dried basil  thyme  basil  dried thyme  minced
          0.090    0.049          0.040      0.027        0.024  0.024  0.024        0.022   0.022
topic: 4
documents:
                          Fried Chicken with Gravy      0.724
 Witchs Finger Bread Sticks with Maple Mustard Dip  0.7033333
                             Manicotti Pancakes II        0.7
3 eggs 1 cup milk 1 cup all-purpose flour----

topic: 0
documents:
                    Mushroom Goat Cheese Pan Sauce  0.5315789
                                Thiebaud Pink Cake  0.5315789
 Poached Seckel Pear with Pomegranate, Cabrales...  0.5229167
 3 cups (750 g/26.4 oz) whole milk 1 cup plus 1 Tbsp (250 g/8.8 oz) heavy cream 3 Tbsp plus 1 tsp (20 g/0.7 oz) Szechuan peppercorns 8 1/2 large (160 g/5.6 oz) egg yolks 3/4 cup plus 2 Tbsp (180 g/6.3 oz) granulated sugar ----- 1/2 pound (227 g) fresh strawberries 1/2 cup (4.3 oz / 120 g) water 1/4 cup (1.8 oz / 50 g) sugar ----- 1/4 cup canned low-sodium chicken broth 1/4 cup full-bodied red wine 
----------------------------------------------------------------------
 broth  chicken broth  celery    low      g  stock  sodium  cups chicken  cup chicken
 0.055          0.041   0.032  0.026  0.023  0.018   0.018         0.016        0.016
topic: 1
documents:
                            Dinengdeng  0.5261905
 Classic Smoked Sausage \u0026 Peppers  0.4745455
                            B